In [1]:
!pip install selenium
!apt-get update

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.6/461.6 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,332 kB]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,047 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,582 kB]
Get:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://pp

In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup

In [38]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument("--no-sandbox")
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920, 1200")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    return driver

In [39]:
driver = web_driver()
driver.get("https://www.youtube.com/@jacksfilms")
# driver.quit()

for _ in range(2):
    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    time.sleep(3)
    print(".")

data = driver.page_source
soup = BeautifulSoup(data, "html.parser")
videos = soup.find_all("div", {"id": "dismissible"})

.
.


In [44]:
print("Total videos: ", len(videos))
master_list = []

for video in videos:
    data_dic = {"title": None, "video_url": None, "views": None, "video_age": None}
    title_data = video.find("a", {"id": "video-title"})
    meta = video.find("div", {"id": "metadata-line"})
    if title_data and meta:
        data_dic["title"] = title_data.text
        data_dic["video_url"] = "https://www.youtube.com" + title_data["href"]
        meta_span = meta.find_all("span")
        if len(meta_span) == 2:
            data_dic["views"] = meta_span[0].text
            data_dic["video_age"] = meta_span[1].text
        # print(data_dic)
        master_list.append(data_dic)
    # break

Total videos:  71


In [ ]:
import pandas as pd
from google.colab import drive

df = pd.DataFrame(master_list)
# print(df.to_string(index=False))
drive.mount("drive")

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).
Data saved to youtube_views


In [57]:
def convert_views(df):
    if df["views"]:
        if "K" in df["views"]:
            views = float(df["views"].split("K")[0]) * 1000
            return views
        elif "M" in df["views"]:
            views = float(df["views"].split("M")[0]) * 1000000
            return views
    else:
        return 0


df["clean_views"] = df.apply(convert_views, axis=1)
df["clean_views"] = df["clean_views"].astype(int)
print(df)

                                                title  \
0             Reading my favorite tweets from YouTube   
1             Reading my favorite tweets from YouTube   
2                 The ACTUALLY GOOD YOUTUBERS Iceberg   
3                I hate jumpscares, so I made an app.   
4   Sssniperwolf came to our home last night. It's...   
..                                                ...   
59                          DUBSTEP SOLVES EVERYTHING   
60                             EPIC HARRY POTTER SONG   
61                                      CENSORED SONG   
62           All I Want for Christmas is a Girlfriend   
63                          48 Hour Film Fest WINNERS   

                                            video_url       views  \
0         https://www.youtube.com/watch?v=vVzA70bWh3g  206K views   
1         https://www.youtube.com/watch?v=vVzA70bWh3g  206K views   
2         https://www.youtube.com/watch?v=-dcuAAMUicw  671K views   
3         https://www.youtube.com/watch

In [58]:
df.to_csv("youtube_views.csv")
!cp youtube_views.csv "drive/My Drive/Colab Notebooks"
# df.to_csv('youtube_views', index=False)
print("Data saved to youtube_views")

Data saved to youtube_views
